In [8]:
# Ranging #
from pypozyx import *
import numpy as np
from pypozyx.definitions.registers import *
import sys
import time
import serial 
from numpy.linalg import inv
from openpyxl import Workbook
write_wb = Workbook()
write_ws = write_wb.active

def tri(zd, H):
    zd = zd.T
    H=H-H[0]
    tmp = zd[0]
    tmp = tmp * tmp
    zd = np.delete(zd,(0), axis=0)
    r1=np.multiply(zd,zd)
    Ha=np.delete(H,(0), axis=0)
    K=np.multiply(Ha,Ha)
    K=K.sum(axis=1)
    b= 0.5*(K-r1+tmp) 
    S_inv = np.linalg.inv(Ha.T*Ha)*Ha.T
    x_hat = S_inv*b
    return x_hat

########################################## RANGING ##############################################
j = 0   
port = get_first_pozyx_serial_port()#'/dev/cu.usbmodem3998326833361'
p = PozyxSerial(port)

remote_id = 0x675a          # the network ID of the remote device
remote = False               # whether to use the given remote device for ranging
if not remote:
    remote_id = None
th = 1000
dt = 1
while True:
    
    H = np.array([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
    H2 = np.matrix([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])         
    zd = np.matrix([zd_1,zd_2,zd_3,zd_4])
    z_meas = zd*10**(-3)
    x_nlos = tri(z_meas*10**(-3),H2)
    x_nlos = x_nlos.T+H[0]
    print(x_nlos)
#################################################################################################
#####################                 Cooperative Localization               ####################
################################################################################################# 
    import numpy as np
    import proxmin

    dX = np.array([1.,0.5])
    radius = 1

    def f(X):
        """Shifted parabola"""
        return np.sum((X - dX)**2, axis=-1)

    def grad_f(X):
        return 2*(X - dX)

    def step_f(X, it=0):
        L = 2. # Lipschitz constant of grad f
        return 1 / L

    def prox_circle(X, step):
        """Projection onto circle"""
        center = np.array([0,0])
        dX = X - center
        # exclude everything other than perimeter of circle
        phi = np.arctan2(dX[1], dX[0])
        return center + radius*np.array([np.cos(phi), np.sin(phi)])

    X = x_nlos # or whereever
    converged, grad, step = proxmin.pgm(X, grad_f, step_f, prox=prox_circle)   
    
########################################################################################################      
#    write_ws['A1'] = 'x'
#    write_ws['B1'] = 'y'
#    write_ws.cell(i+1,1,x_nlos[(0,0)])
#    write_ws.cell(i+1,2,x_nlos[(0,1)])
    write_ws.cell(j+1,1,z_meas.item(0))
    write_ws.cell(j+1,2,z_meas.item(1))
    write_ws.cell(j+1,3,z_meas.item(2))
    write_ws.cell(j+1,4,z_meas.item(3))
    #write_ws.cell(j+1,3,x_hat[0])
    #write_ws.cell(j+1,4,x_hat[1])
    write_wb.save('/Users/swbaek/Desktop/ranging_2.xlsx')
    j = j+1

PozyxConnectionError: Couldn't connect to Pozyx, AttributeError: 'NoneType' object has no attribute 'description'

In [5]:
pip install pypozyx

  Using cached pypozyx-1.3.0-py2.py3-none-any.whl (51 kB)
  Using cached pyserial-3.4-py2.py3-none-any.whl (193 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pyserial

In [7]:
print(z_meas.item(0))

0.0
